In [2]:
import pandas as pd
import json


filename = "race_data_168096594.json"

In [5]:
records = []


def try_parse_json(string):
    try:
        return json.loads(string), None
    except json.JSONDecodeError as e:
        return None, e


with open("race_data_168096594.json", "r") as file:
    buffer = ""
    for line in file:
        buffer += line.strip()
        parsed_json, error = try_parse_json(buffer)
        if parsed_json:
            buffer = ""  # Clear buffer if successful parse
            if parsed_json["op"] == "mcm":
                pt = parsed_json["pt"]
                for mc_item in parsed_json.get("mc", []):
                    market_id = mc_item["id"]
                    for rc in mc_item.get("rc", []):
                        if "batb" in rc and len(rc["batb"]) > 1:
                            best_available_to_back = rc["batb"][1][1]
                        else:
                            best_available_to_back = None
                        ltp = rc.get("ltp")
                        tv = rc.get("tv")
                        horse_id = rc["id"]
                        records.append(
                            {
                                "timestamp_unix": pt,
                                "market_id": market_id,
                                "horse_id": horse_id,
                                "best_available_to_back": best_available_to_back,
                                "last_traded_price": ltp,
                                "traded_volume": tv,
                            }
                        )
        elif error and "Extra data" in str(
            error
        ):  # Possible end of one object and start of another
            first_part, _, buffer = buffer.partition("}{")
            buffer = "{" + buffer  # Assume the split is between separate JSON objects
            first_part += "}"
            try_parse_json(first_part)  # Retry parsing the first part
        # Handle other types of JSONDecodeError if necessary

In [6]:
# Create a DataFrame from the records
df = pd.DataFrame(records)


In [9]:
df.tail(1000)

,timestamp_unix,market_id,horse_id,best_available_to_back,last_traded_price,traded_volume
34336,1580573868503,1.168096594,22972635,9.80,10.50,20585.65
34337,1580573868503,1.168096594,16927442,NaN,22.00,12517.58
34338,1580573868503,1.168096594,22101589,3.95,4.00,229729.74
34339,1580573868503,1.168096594,23405261,29.00,30.00,7551.54
34340,1580573868503,1.168096594,21848159,9.80,13.50,76375.33
...,...,...,...,...,...,...
35331,1580573985099,1.168096594,16927442,75.00,1000.00,13256.69
35332,1580573985099,1.168096594,22101589,NaN,1.01,320188.44
35333,1580573985099,1.168096594,23405261,8.00,1000.00,9617.63
35334,1580573985099,1.168096594,21848159,10.00,1000.00,78481.91


In [10]:
# describe the dataframe

df.describe()


,timestamp_unix,horse_id,best_available_to_back,last_traded_price,traded_volume
count,3.533600e+04,3.533600e+04,12138.000000,35336.000000,35336.000000
mean,1.580556e+12,2.015543e+07,11.731981,16.002382,14500.115202
std,2.399994e+07,2.969125e+06,18.251229,59.155546,30019.736814
min,1.580469e+12,1.476535e+07,1.010000,0.000000,0.000000
25%,1.580549e+12,1.759620e+07,5.000000,5.700000,1262.590000
50%,1.580568e+12,2.184816e+07,8.400000,8.800000,4210.810000
75%,1.580573e+12,2.210159e+07,13.500000,15.500000,14681.737500
max,1.580574e+12,2.340526e+07,650.000000,1000.000000,320188.440000


In [11]:
df.shape

(35336, 6)